# Kaggle - WiDS Datathon 2022

## Modelo Nº1

### Description:

- All the features
- The entire dataset for train (not separation train/test)
- Catboost
- Hiperparameters by default
- Long value of iterations: 10000

- Public Score = 42.525

# 1. Modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

#model
from catboost import CatBoostRegressor

# 2. Data

In [2]:
#paths to the Github repo

train_path = 'https://raw.githubusercontent.com/ccollado7/wds-datathon-2022/main/data/raw/train.csv'
test_path = 'https://raw.githubusercontent.com/ccollado7/wds-datathon-2022/main/data/raw/test.csv'
submit_path = 'https://raw.githubusercontent.com/ccollado7/wds-datathon-2022/main/data/raw/sample_solution.csv'

In [3]:
#train dataset

train = pd.read_csv(train_path)
train.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,248.682615,0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,26.500150,1
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,24.693619,2
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,48.406926,3
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,3.899395,4


In [4]:
#test dataset

test = pd.read_csv(test_path)
test.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_below_0F,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,id
0,7,State_1,Commercial,Grocery_store_or_food_market,28484.0,1994.0,37.0,2.4,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75757
1,7,State_1,Commercial,Grocery_store_or_food_market,21906.0,1961.0,55.0,45.7,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75758
2,7,State_1,Commercial,Grocery_store_or_food_market,16138.0,1950.0,1.0,59.1,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75759
3,7,State_1,Commercial,Grocery_store_or_food_market,97422.0,1971.0,34.0,35.4,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75760
4,7,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,35.0,1.8,38,50.596774,...,0,29,5,2,0,340.0,330.0,22.8,126.0,75761


In [5]:
#sumbit

submit = pd.read_csv(submit_path)
submit.head()

,id,site_eui
0,75757,0.0
1,75758,0.0
2,75759,0.0
3,75760,0.0
4,75761,0.0


In [6]:
x_train = train.drop(['id','site_eui'], axis=1)
x_train.columns

Index(['Year_Factor', 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION',
       'january_min_temp', 'january_avg_temp', 'january_max_temp',
       'february_min_temp', 'february_avg_temp', 'february_max_temp',
       'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp',
       'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp',
       'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp',
       'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp',
       'august_avg_temp', 'august_max_temp', 'september_min_temp',
       'september_avg_temp', 'september_max_temp', 'october_min_temp',
       'october_avg_temp', 'october_max_temp', 'november_min_temp',
       'november_avg_temp', 'november_max_temp', 'december_min_temp',
       'december_avg_temp', 'december_max_temp', 'cooling_degree_days',
       'heating_degree_days', 'precipitation_inches', 'snowfall_inc

In [7]:
y_train = train['site_eui']

In [8]:
cat_columns = list(train.select_dtypes(include=['object']).columns)
cat_columns

['State_Factor', 'building_class', 'facility_type']

In [16]:
model_1_1 = CatBoostRegressor(cat_features = cat_columns,
                             iterations=10000,
                             verbose=10)

In [17]:
model_1_1.fit(x_train,y_train,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.012478
0:	learn: 58.0687857	total: 45.2ms	remaining: 7m 32s
10:	learn: 56.3577087	total: 329ms	remaining: 4m 58s
20:	learn: 54.8928309	total: 586ms	remaining: 4m 38s
30:	learn: 53.6897253	total: 916ms	remaining: 4m 54s
40:	learn: 52.7253768	total: 1.24s	remaining: 5m 1s
50:	learn: 51.8350110	total: 1.51s	remaining: 4m 54s
60:	learn: 51.0809293	total: 1.83s	remaining: 4m 57s
70:	learn: 50.4485177	total: 2.16s	remaining: 5m 2s
80:	learn: 49.9169226	total: 2.49s	remaining: 5m 4s
90:	learn: 49.4770272	total: 2.78s	remaining: 5m 2s
100:	learn: 49.0620484	total: 3.08s	remaining: 5m 1s
110:	learn: 48.7331811	total: 3.43s	remaining: 5m 5s
120:	learn: 48.4346738	total: 3.7s	remaining: 5m 2s
130:	learn: 48.1514191	total: 4.03s	remaining: 5m 3s
140:	learn: 47.8903450	total: 4.43s	remaining: 5m 9s
150:	learn: 47.6298505	total: 4.81s	remaining: 5m 13s
160:	learn: 47.4348182	total: 5.18s	remaining: 5m 16s
170:	learn: 47.2512097	total: 5.56s	remaining: 5m 19s
180:	learn: 47.064

1520:	learn: 42.2997882	total: 48.3s	remaining: 4m 29s
1530:	learn: 42.2872421	total: 48.7s	remaining: 4m 29s
1540:	learn: 42.2748549	total: 48.9s	remaining: 4m 28s
1550:	learn: 42.2632650	total: 49.3s	remaining: 4m 28s
1560:	learn: 42.2491390	total: 49.6s	remaining: 4m 28s
1570:	learn: 42.2358497	total: 49.9s	remaining: 4m 27s
1580:	learn: 42.2190272	total: 50.2s	remaining: 4m 27s
1590:	learn: 42.2040182	total: 50.5s	remaining: 4m 27s
1600:	learn: 42.1959489	total: 50.8s	remaining: 4m 26s
1610:	learn: 42.1849547	total: 51.1s	remaining: 4m 26s
1620:	learn: 42.1713587	total: 51.4s	remaining: 4m 25s
1630:	learn: 42.1605523	total: 51.6s	remaining: 4m 24s
1640:	learn: 42.1445867	total: 51.9s	remaining: 4m 24s
1650:	learn: 42.1335094	total: 52.2s	remaining: 4m 24s
1660:	learn: 42.1224890	total: 52.5s	remaining: 4m 23s
1670:	learn: 42.1114236	total: 52.8s	remaining: 4m 23s
1680:	learn: 42.0986325	total: 53.1s	remaining: 4m 22s
1690:	learn: 42.0898237	total: 53.4s	remaining: 4m 22s
1700:	lear

3010:	learn: 40.7858650	total: 1m 37s	remaining: 3m 45s
3020:	learn: 40.7769898	total: 1m 37s	remaining: 3m 45s
3030:	learn: 40.7695779	total: 1m 37s	remaining: 3m 45s
3040:	learn: 40.7596284	total: 1m 38s	remaining: 3m 45s
3050:	learn: 40.7530936	total: 1m 38s	remaining: 3m 44s
3060:	learn: 40.7405599	total: 1m 38s	remaining: 3m 44s
3070:	learn: 40.7315579	total: 1m 39s	remaining: 3m 44s
3080:	learn: 40.7212119	total: 1m 39s	remaining: 3m 43s
3090:	learn: 40.7159036	total: 1m 40s	remaining: 3m 43s
3100:	learn: 40.7088105	total: 1m 40s	remaining: 3m 43s
3110:	learn: 40.7029183	total: 1m 40s	remaining: 3m 43s
3120:	learn: 40.6965351	total: 1m 41s	remaining: 3m 42s
3130:	learn: 40.6902035	total: 1m 41s	remaining: 3m 42s
3140:	learn: 40.6792931	total: 1m 41s	remaining: 3m 42s
3150:	learn: 40.6672010	total: 1m 42s	remaining: 3m 42s
3160:	learn: 40.6571513	total: 1m 42s	remaining: 3m 42s
3170:	learn: 40.6516011	total: 1m 42s	remaining: 3m 41s
3180:	learn: 40.6418844	total: 1m 43s	remaining:

4490:	learn: 39.6387676	total: 2m 29s	remaining: 3m 3s
4500:	learn: 39.6332568	total: 2m 29s	remaining: 3m 2s
4510:	learn: 39.6266770	total: 2m 29s	remaining: 3m 2s
4520:	learn: 39.6184675	total: 2m 30s	remaining: 3m 2s
4530:	learn: 39.6112673	total: 2m 30s	remaining: 3m 1s
4540:	learn: 39.6064935	total: 2m 30s	remaining: 3m 1s
4550:	learn: 39.5994685	total: 2m 31s	remaining: 3m 1s
4560:	learn: 39.5948773	total: 2m 31s	remaining: 3m
4570:	learn: 39.5863537	total: 2m 31s	remaining: 3m
4580:	learn: 39.5793048	total: 2m 32s	remaining: 3m
4590:	learn: 39.5722803	total: 2m 32s	remaining: 2m 59s
4600:	learn: 39.5673954	total: 2m 32s	remaining: 2m 59s
4610:	learn: 39.5630471	total: 2m 33s	remaining: 2m 58s
4620:	learn: 39.5597062	total: 2m 33s	remaining: 2m 58s
4630:	learn: 39.5562317	total: 2m 33s	remaining: 2m 58s
4640:	learn: 39.5511162	total: 2m 34s	remaining: 2m 57s
4650:	learn: 39.5469626	total: 2m 34s	remaining: 2m 57s
4660:	learn: 39.5427148	total: 2m 34s	remaining: 2m 57s
4670:	learn

5970:	learn: 38.7958463	total: 3m 18s	remaining: 2m 13s
5980:	learn: 38.7899475	total: 3m 18s	remaining: 2m 13s
5990:	learn: 38.7842657	total: 3m 18s	remaining: 2m 13s
6000:	learn: 38.7803002	total: 3m 19s	remaining: 2m 12s
6010:	learn: 38.7743097	total: 3m 19s	remaining: 2m 12s
6020:	learn: 38.7704563	total: 3m 20s	remaining: 2m 12s
6030:	learn: 38.7661696	total: 3m 20s	remaining: 2m 11s
6040:	learn: 38.7614491	total: 3m 20s	remaining: 2m 11s
6050:	learn: 38.7560451	total: 3m 20s	remaining: 2m 11s
6060:	learn: 38.7510678	total: 3m 21s	remaining: 2m 10s
6070:	learn: 38.7449899	total: 3m 21s	remaining: 2m 10s
6080:	learn: 38.7401870	total: 3m 22s	remaining: 2m 10s
6090:	learn: 38.7357920	total: 3m 22s	remaining: 2m 9s
6100:	learn: 38.7318046	total: 3m 22s	remaining: 2m 9s
6110:	learn: 38.7243875	total: 3m 23s	remaining: 2m 9s
6120:	learn: 38.7205274	total: 3m 23s	remaining: 2m 8s
6130:	learn: 38.7143306	total: 3m 23s	remaining: 2m 8s
6140:	learn: 38.7065646	total: 3m 24s	remaining: 2m 8

7450:	learn: 38.0239935	total: 4m 11s	remaining: 1m 26s
7460:	learn: 38.0184649	total: 4m 12s	remaining: 1m 25s
7470:	learn: 38.0131693	total: 4m 12s	remaining: 1m 25s
7480:	learn: 38.0078800	total: 4m 12s	remaining: 1m 25s
7490:	learn: 38.0038295	total: 4m 13s	remaining: 1m 24s
7500:	learn: 38.0002578	total: 4m 13s	remaining: 1m 24s
7510:	learn: 37.9955748	total: 4m 13s	remaining: 1m 24s
7520:	learn: 37.9906261	total: 4m 14s	remaining: 1m 23s
7530:	learn: 37.9858506	total: 4m 14s	remaining: 1m 23s
7540:	learn: 37.9821060	total: 4m 14s	remaining: 1m 23s
7550:	learn: 37.9783342	total: 4m 15s	remaining: 1m 22s
7560:	learn: 37.9734431	total: 4m 15s	remaining: 1m 22s
7570:	learn: 37.9702652	total: 4m 16s	remaining: 1m 22s
7580:	learn: 37.9654153	total: 4m 16s	remaining: 1m 21s
7590:	learn: 37.9620140	total: 4m 16s	remaining: 1m 21s
7600:	learn: 37.9562437	total: 4m 17s	remaining: 1m 21s
7610:	learn: 37.9527751	total: 4m 17s	remaining: 1m 20s
7620:	learn: 37.9461049	total: 4m 17s	remaining:

8940:	learn: 37.3763533	total: 5m 5s	remaining: 36.2s
8950:	learn: 37.3710200	total: 5m 6s	remaining: 35.9s
8960:	learn: 37.3680502	total: 5m 6s	remaining: 35.5s
8970:	learn: 37.3613499	total: 5m 6s	remaining: 35.2s
8980:	learn: 37.3576554	total: 5m 7s	remaining: 34.9s
8990:	learn: 37.3547596	total: 5m 7s	remaining: 34.5s
9000:	learn: 37.3520545	total: 5m 7s	remaining: 34.2s
9010:	learn: 37.3473695	total: 5m 8s	remaining: 33.9s
9020:	learn: 37.3432087	total: 5m 8s	remaining: 33.5s
9030:	learn: 37.3405352	total: 5m 9s	remaining: 33.2s
9040:	learn: 37.3365518	total: 5m 9s	remaining: 32.9s
9050:	learn: 37.3335409	total: 5m 10s	remaining: 32.5s
9060:	learn: 37.3296664	total: 5m 10s	remaining: 32.2s
9070:	learn: 37.3255171	total: 5m 10s	remaining: 31.8s
9080:	learn: 37.3214710	total: 5m 11s	remaining: 31.5s
9090:	learn: 37.3182925	total: 5m 11s	remaining: 31.1s
9100:	learn: 37.3157536	total: 5m 11s	remaining: 30.8s
9110:	learn: 37.3124313	total: 5m 12s	remaining: 30.5s
9120:	learn: 37.30854

In [18]:
x_test = test.drop('id', axis=1)
x_test.columns

Index(['Year_Factor', 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION',
       'january_min_temp', 'january_avg_temp', 'january_max_temp',
       'february_min_temp', 'february_avg_temp', 'february_max_temp',
       'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp',
       'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp',
       'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp',
       'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp',
       'august_avg_temp', 'august_max_temp', 'september_min_temp',
       'september_avg_temp', 'september_max_temp', 'october_min_temp',
       'october_avg_temp', 'october_max_temp', 'november_min_temp',
       'november_avg_temp', 'november_max_temp', 'december_min_temp',
       'december_avg_temp', 'december_max_temp', 'cooling_degree_days',
       'heating_degree_days', 'precipitation_inches', 'snowfall_inc

In [19]:
submit['site_eui'] = model_1_1.predict(x_test)
submit.to_csv('modelo_1_1.csv',index=False)